# 1. Global Variables of the experiment


In [1]:

import itertools
import csv

In [7]:
# step 2 specify geometry
stl_path = '/Users/norapusok/slicer/models/20mm_cube.stl'
###############################
# step 2a parametric geometry
cube_side = '20 mm'
###############################
# step 3 slicer parameters

# place sliced files ready to be printed in dir
output_sliced = u"/Users/norapusok/slicer/sliced_files/"
path_slicer_exec = '/Applications/PrusaSlicer.app/Contents/MacOS/PrusaSlicer'

# label names based on concat "L" + 0,1,2....
# N - max nr. of output files to print
def inf_labels(N=100): return list( map(lambda s1, s2: s1 + s2, itertools.repeat("L"), map(str, range(N))) )
glob_labels = []

labelled_meshes = u"/Users/norapusok/slicer/labelled_meshes/"
#slicer_input_meshes = ["L0.stl", "L1.stl", "L2.stl", "L3.stl"]
#def input_meshes(mlabels): return list( map(lambda s1, s2: s1 + s2, mlabels, repeat(".stl")) )
# usage, max nr of prints as par
# labels = inf_labels(1000)
# input_meshes(labels)

# dict key is - parameter name from config file for the optionals
# specify value ranges - termianl search /Applications/PrusaSlicer.app/Contents/MacOS/PrusaSlicer --help-fff
dict_slicer_pars = {        
         'layer_height':('0.01', '0.3') 
         ,'fill_pattern':['stars', 'concentric']
        }
# specify number of prints of each unique parameter set.
duplicate_print = 1
output_configs = u"/Users/norapusok/slicer/output_configs/"

# in Prusa slicer GUI select printer fx. Original Prusa i3 MK3, 
#                     select material if none Generic PLA/PETG/ABS
#      than File/Export/Export config... to obtain config.ini, change path below
config_pla = u"/Users/norapusok/slicer/config_prusa/config.ini"
#config_petg = u"/Users/norapusok/slicer/config_prusa/config_petg.ini"

# in config.ini file insert first row [DEFAULT] (for parserlibrary to parse) 
#                    find the parameter you want to vary fx. fill_pattern & layer_height in the tuple
#                  or change value of par in GUI expert settings and export config_bundle to expose par name.
# pars 

###############################          

# step 4  manual printing
# step 5 measurements csv
# step 6 TEA

# 3. Slicing 

In [8]:
# model of comibining parameters from dict_slicer_pars: Cartesian product
list_par_names = tuple(dict_slicer_pars.keys())
list_vals = list(itertools.product(*dict_slicer_pars.values())) # itertools starmap ? 

#hey = [list(zip(list_par_names, val)) for val in list_vals]
#print(hey[1])
print(list(list_par_names))
print(list_vals)


['layer_height', 'fill_pattern']
[('0.01', 'stars'), ('0.01', 'concentric'), ('0.3', 'stars'), ('0.3', 'concentric')]


In [9]:
import configparser #https://docs.python.org/3/library/configparser.html
import io
# parlist [(par1,val1), (par2,val2), ....]
#def change_config_par(parlist, config):
#    for (par,val) in parlist:
#        config['DEFAULT'][par]=val
#    return config
    

# def make_config(list_par_names, list_vals, reps=duplicate_print):    
# zip to format [(par1,val1), (par2,val2), ....] so per print nr. of pars can differ
zipped = [list(zip(list_par_names, val)) for val in list_vals]
    
# labels
glob_labels = inf_labels(N=len(zipped)*duplicate_print)
labels = inf_labels(N=len(zipped)*duplicate_print)
# counter
c = 0
    
# write log of sliced file data to csv
with open("output_slicing.csv", "a") as fp:
    wr = csv.writer(fp, delimiter='\t')
    # foreach (par,val)
    for partuple in zipped:
        # repeat for #duplicate_print
        for _reps in range(duplicate_print):
             
                # parse input config
                config = configparser.ConfigParser()
                config.read(config_pla) # potentially change for other filament

                #replace par=val in parsed config
                for (par,val) in partuple:
                    config['DEFAULT'][par]=val
                
                # open config-<labelname> write
                with open(output_configs+ "/config-" + labels[c] + ".ini", 'w') as out:
                    
                    #remove the manually added section header 
                    #https://stackoverflow.com/questions/66137056/how-to-write-ini-files-without-sections
                    buf = io.StringIO()
                    config.write(buf)
                    buf.seek(0)
                    next(buf) #skip first line with [DEFAULT] 
                    
                    # write to file without section header
                    out.write(buf.read())
                
                # add label & (par_i,val_i) .. to log
                wr.writerow([labels[c]] + partuple)  
                
                # print logged lines to console
                print([labels[c]] + partuple)
                
                # increment labels pointer
                c = c+1
# print labels sequence                
print(glob_labels)

['L0', ('layer_height', '0.01'), ('fill_pattern', 'stars')]
['L1', ('layer_height', '0.01'), ('fill_pattern', 'concentric')]
['L2', ('layer_height', '0.3'), ('fill_pattern', 'stars')]
['L3', ('layer_height', '0.3'), ('fill_pattern', 'concentric')]
['L0', 'L1', 'L2', 'L3']


In [10]:
# https://github.com/valkyriesavage/objectify
import subprocess

def slice_mesh(custom_stl_path, custom_config_path, output_dir):
    slice_command = [path_slicer_exec,
                    '--load', custom_config_path, 
                    '--export-gcode', custom_stl_path,
                    '--output', output_dir]
    results = subprocess.check_output(slice_command)
    
    # the last line from Prusa Slicer is "Slicing result exported to ..."
    last_line = results.decode('utf-8').strip().split("\n")[-1]
    location = last_line.split(" exported to ")[1]

    return location

### for now manually check that meshes match with labels in dir.

In [11]:


for label in labels:
    # custom paths for each slicer CLI command in slice_mesh
    labelled_stl = labelled_meshes + label + ".stl"
    config_path = '/Users/norapusok/slicer/output_configs/config-' + label + ".ini"
    sliced_fname = output_sliced + label +".gcode"
    print(label)
    print(labelled_stl)
    print(config_path)
    print(sliced_fname)
    slice_mesh(labelled_stl, config_path, sliced_fname)

L0
/Users/norapusok/slicer/labelled_meshes/L0.stl
/Users/norapusok/slicer/output_configs/config-L0.ini
/Users/norapusok/slicer/sliced_files/L0.gcode
L1
/Users/norapusok/slicer/labelled_meshes/L1.stl
/Users/norapusok/slicer/output_configs/config-L1.ini
/Users/norapusok/slicer/sliced_files/L1.gcode
L2
/Users/norapusok/slicer/labelled_meshes/L2.stl
/Users/norapusok/slicer/output_configs/config-L2.ini
/Users/norapusok/slicer/sliced_files/L2.gcode
L3
/Users/norapusok/slicer/labelled_meshes/L3.stl
/Users/norapusok/slicer/output_configs/config-L3.ini
/Users/norapusok/slicer/sliced_files/L3.gcode


### confirm gcodes in sliced_files dir.

# 2a. Or create parametric geometry and turn it into a mesh
### modify side

In [ ]:
#brew install --cask freecad
import sys
sys.path.append('/Applications/FreeCAD.app/Contents/Resources/lib') 
import FreeCAD, Part #import FreeCADGui

# pip install PySide2
import Mesh
import Draft

In [ ]:
# FreeCAD create cube    
def makeBox(side = '20 mm'):
    App.newDocument()
    App.ActiveDocument.addObject("Part::Box","Box")
    App.ActiveDocument.ActiveObject.Label = "Cube"
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Width = side
    FreeCAD.ActiveDocument.getObject('Box').Length = side
    FreeCAD.ActiveDocument.getObject('Box').Height = side
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Shape.exportStl('box.stl')
# export as stl https://gist.github.com/hyOzd/2e75a9816cfabeb5b4aa

# Add label
def makeLabel(label_string="L0"):
    # https://wiki.freecad.org/Draft_ShapeString_tutorial
    ss=Draft.make_shapestring(String=label_string, FontFile="/Library/Fonts/DejaVuSans.ttf", Size=5.0, Tracking=0.0)
    plm=FreeCAD.Placement()
    plm.Base=FreeCAD.Vector(4.0, 4.0, side=20)
    plm.Rotation.Q=(0.0, 0.0, 0, 1.0)
    ss.Placement=plm
    ss.Support=None
    Draft.autogroup(ss)
    FreeCAD.ActiveDocument.recompute()
    
    App.ActiveDocument.addObject('Part::Extrusion','Extrude')
    f = App.ActiveDocument.getObject('Extrude')
    f.Base = App.ActiveDocument.getObject('ShapeString')
    f.DirMode = "Normal"
    f.DirLink = None
    f.LengthFwd = 5.000000000000000
    f.LengthRev = 0.000000000000000
    f.Solid = False
    f.Reversed = False
    f.Symmetric = False
    f.TaperAngle = 0.000000000000000
    f.TaperAngleRev = 0.000000000000000
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('ShapeString').Visibility = False
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('Extrude').Placement = App.Placement(App.Vector(2,2,0),App.Rotation(App.Vector(0,0,1),0))
    App.ActiveDocument.recompute()
    #FreeCAD.ActiveDocument.getObject('Extrude').Shape.exportStl('ext.stl')
    __objs__ = []
    __objs__.append(FreeCAD.ActiveDocument.getObject("Box"))
    __objs__.append(FreeCAD.ActiveDocument.getObject("Extrude"))
    Mesh.export(__objs__, u"/Users/norapusok/slicer/" + "comb_" + label_string + ".stl")


#### For now add these handwritten labels

In [ ]:
makeBox()
for l in ["L0", "L1", "L2", "L3"]:
    makeLabel(label_string=l)
# or stl merge https://pypi.org/project/numpy-stl/

### View combi.stl in Prusa

# 4. Go print gcode files manually. 


In [ ]:
# or send to simplyprint.io for later

# 5. Take measurements using a caliper. Show .csv

# 6. NHST null hypothesis significance testing 


### Automatically select and execute statistical test using TEA

In [ ]:
import subprocess
# pip install tealang
import tea

##### 1/5 Dataset (rows are treated as individual observations, i.e. all variables are between-subjects.)

In [ ]:
data_path = "./cube_observations.csv"

##### 2/5 Variables 

#### Data types short help
##### nominal (discrete categories)
##### ordinal (discrete ordered categories)
##### numeric (continuous values!, range spec. fx [0,1] is optional)
##### - interval scale: real-additive (+)&(-) only valid ops.; 0-only a value (does not indicate lack of value); fx temperature/12H clock
##### - ratio scale: real-multiplicative (*)&(/)&(mean/median) also valid. ops.; 0 spec. elem fx. object's physical measurements 
      think binomial vs count (discrete)


In [ ]:

variables = [
    {
        'name' : 'layer height',
        'data type' : 'ratio'
        # ,'range' : [0.08, 0.28] #optional par.
    },
    {
        'name' : 'material',
        'data type' : 'nominal',
        'categories' : ['PLA', 'PETG', 'ABS']    # , 'nylon', 'metal']
    },
    {
        'name' : 'shrinkage x-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'shrinkage y-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'shrinkage z-axis',
        'data type' : 'ratio'
    }
        
# other variables infill pattern, travel speed?
]

##### 3/5 Study design (describe independent variables and dependent variables. multiple hypothesis?)

In [ ]:
study_design = {
    'study type': 'experiment',
    'independent variables': ['layer height', 'material'],
    'dependent variables': ['shrinkage x-axis', 'shrinkage y-axis', 'shrinkage z-axis']
}

##### 4/5 (optional )Assumptions (explicitely provide domain knowledge - relevant to testing your hypothesis)

In [ ]:
# physical phenomena usually normally distributed. 
#assumptions = {
#    'normal distribution': ['shrinkage x-axis', 'shrinkage y-axis', 'shrinkage z-axis']
#}
#tea.assume(assumptions, 'strict')   # mode relaxed(enforce user's choice) / strict(override)

##### 5/5 Hypothesis

In [ ]:
# example usage 
tea.data(data_path) # no "key" column provided, so between subject study.  # (data_path, key='ID')
                    # Exactly one observation per print parameter settings
tea.define_variables(variables)
tea.define_study_design(study_design)
tea.hypothesize(['material', 'shrinkage z-axis' ], ['material:PETG > ABS'])

In [ ]:
"""
Results:
--------------
Test: kruskall_wallis
***Test assumptions:
Independent (not paired) observations: material
Exactly one explanatory variable: material
Exactly one explained variable: shrinkage x-axis
Continuous (not categorical) data: shrinkage x-axis
Variable is categorical: material
Variable has two or more categories: material

***Test results:
name = Kruskall Wallis
test_statistic = 9.25461
p_value = 0.00978
adjusted_p_value = 0.00978
alpha = 0.05
dof = 4
Null hypothesis = There is no difference in medians between material = PLA, PETG, ABS on shrinkage x-axis.
Interpretation = t(4) = 9.25461, p = 0.00978. Reject the null hypothesis at alpha = 0.05. There is a difference in medians of shrinkage x-axis for at least one of material = PLA, PETG, ABS.

"""